In [1]:
%matplotlib inline

import pandas as pd
import matplotlib.pyplot as plt
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
import string

# Importing Gensim
import gensim
from gensim import corpora

import sys
reload(sys)
sys.setdefaultencoding('utf-8')

stop = set(stopwords.words('english'))
exclude = set(string.punctuation) 
lemma = WordNetLemmatizer()
doc_complete = []

def clean(doc):
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    return normalized

Read data into a DataFrame

In [3]:
data = pd.read_json('articles.json')

This will show all the column names

In [4]:
data.columns

Index([u'_id', u'abstract', u'blog', u'byline', u'document_type', u'headline',
       u'keywords', u'lead_paragraph', u'multimedia', u'news_desk',
       u'print_page', u'pub_date', u'section_name', u'snippet', u'source',
       u'subsection_name', u'type_of_material', u'web_url', u'word_count'],
      dtype='object')

Sample article data

In [6]:
data.head(20)

,_id,abstract,blog,byline,document_type,headline,keywords,lead_paragraph,multimedia,news_desk,print_page,pub_date,section_name,snippet,source,subsection_name,type_of_material,web_url,word_count
0,5176fb9acf28d02a61002a45,None,[],"{u'person': [{u'organization': u'', u'role': u...",article,{u'main': u'Boomers and Divorce: Share Your Ex...,"[{u'rank': u'3', u'is_major': u'N', u'value': ...",Booming is looking for baby boomer couples who...,"[{u'width': 190, u'url': u'images/2013/08/09/b...",Styles,None,2013-08-09T00:00:00Z,Booming,Booming is looking for baby boomer couples who...,The New York Times,None,News,http://www.nytimes.com/2013/04/22/booming/boom...,163
1,51d5aa507e0d9c0839d2e008,Ligaya Mishan reviews restaurant Nightingale 9...,[],"{u'contributor': u'', u'person': [{u'organizat...",article,{u'seo': u'Hungry City: Nightingale 9 in Carro...,"[{u'rank': u'3', u'is_major': u'N', u'value': ...","Nightingale 9, in Carroll Gardens, Brooklyn, b...","[{u'width': 190, u'url': u'images/2013/07/10/d...",Dining,8,2013-07-10T00:00:00Z,Dining & Wine,"Nightingale 9, in Carroll Gardens, Brooklyn, b...",The New York Times,Dining & Wine,Review,http://www.nytimes.com/2013/07/10/dining/revie...,681
2,51d5c9267e0d9c0839d2e04e,Eric Asimov The Pour column extolls virtues of...,[],"{u'contributor': u'', u'person': [{u'organizat...",article,"{u'main': u'On the Scent of Summer', u'kicker'...","[{u'rank': u'1', u'is_major': u'Y', u'value': ...","Muscat wines from Alsace can be dry, slightly ...","[{u'width': 190, u'url': u'images/2013/07/10/d...",Dining,9,2013-07-10T00:00:00Z,Dining & Wine,"Muscat wines from Alsace can be dry, slightly ...",The New York Times,None,News,http://www.nytimes.com/2013/07/10/dining/on-th...,948
3,51d6db467e0d9c0839d2e264,None,[],"{u'contributor': u'', u'person': [{u'organizat...",article,"{u'main': u'Inside the List', u'print_headline...","[{u'rank': u'1', u'is_major': u'N', u'value': ...","It’s wedding season, and Elin Hilderbrand’s “B...","[{u'width': 190, u'url': u'images/2013/07/07/b...",BookReview,22,2013-07-14T00:00:00Z,Books,"It’s wedding season, and Elin Hilderbrand’s “B...",The New York Times,Sunday Book Review,News,http://www.nytimes.com/2013/07/14/books/review...,609
4,51d6f2f97e0d9c0839d2e2c0,None,[],"{u'contributor': u'', u'person': [{u'firstname...",article,{u'main': u'Pan-Cooked Chicken Scaloppine With...,"[{u'rank': u'3', u'is_major': u'N', u'value': ...",Roasted apricots go well with savory dishes li...,"[{u'width': 190, u'url': u'images/2013/07/06/s...",Science,,2013-07-10T00:00:00Z,Health,Roasted apricots go well with savory dishes li...,The New York Times,None,Recipe,http://www.nytimes.com/2013/07/10/health/pan-c...,625
5,51d6fe5f7e0d9c0839d2e2ea,David Tanis City Kitchen column features recip...,[],"{u'contributor': u'', u'person': [{u'organizat...",article,"{u'main': u'Wild Salmon Is Worth the Price', u...","[{u'rank': u'1', u'is_major': u'N', u'value': ...","It’s not cheap, but the flavor is incredible. ...","[{u'width': 190, u'url': u'images/2013/07/10/d...",Dining,3,2013-07-10T00:00:00Z,Dining & Wine,"It’s not cheap, but the flavor is incredible. ...",The New York Times,None,News,http://www.nytimes.com/2013/07/10/dining/wild-...,432
6,51d728f87e0d9c0839d2e372,Melissa Clark A Good Appetite column gives sug...,[],"{u'contributor': u'', u'person': [{u'organizat...",article,{u'main': u'A Minimalist Approach to Cooking o...,"[{u'rank': u'1', u'is_major': u'Y', u'value': ...","Cooking in a rental-house kitchen, with fewer ...","[{u'width': 190, u'url': u'images/2013/07/10/d...",Dining,1,2013-07-10T00:00:00Z,Dining & Wine,"Cooking in a rental-house kitchen, with fewer ...",The New York Times,None,News,http://www.nytimes.com/2013/07/10/dining/a-min...,891
7,51da116a7e0d9c0839d2e784,"Snoo Wilson, British playwright and director w...",[],"{u'contributor': u'', u'person': [{u'firstname...",article,"{u'main': u'Snoo Wilson, Surrealistic British ...","[{u'rank': u'3', u'is_major': u'Y', u'value': ...",Mr. Wils